# Avaliação do modelo YOLOv8

Este notebook carrega um modelo YOLOv8 treinado, executa a validação no *split* escolhido (val/test/train) e gera as métricas e gráficos (PR curve, Confusion Matrix, etc.).

⚠️ **Pré‑requisitos**: ter o pacote `ultralytics` instalado e o ficheiro de pesos `.pt`, bem como o `data.yaml` correspondente.

In [1]:
!pip install ultralytics

In [4]:
# ⬅️ Edite aqui os caminhos antes de correr
weights_path = 'runs/content/runs/detect/yolov8m_chest_v2_focal_colab/weights/best.pt'  # caminho para .pt treinado
data_yaml     = 'chestx_det10_yolo/data.yaml'                    # caminho para data.yaml
split         = 'val'                               # 'train', 'val' ou 'test'
batch_size    = 16
img_size      = 640


In [5]:
from ultralytics import YOLO

model = YOLO(weights_path)

metrics = model.val(
    data=data_yaml,
    split=split,
    imgsz=img_size,
    batch=batch_size,
    plots=True,      # gera PR curve, confusion matrix, etc.
    save_hybrid=False,
)

print('\n=== Métricas de avaliação ===')
for k, v in metrics.items():
    print(f'{k}: {v}')


WARNING 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.162  Python-3.12.7 torch-2.7.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 92 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 1.62.8 ms, read: 31.83.0 MB/s, size: 384.2 KB)


val: Scanning C:\Users\lynxv\Desktop\Faculdade\Mestrado\2º Semestre\APVC\ProjectV2\chestx_det10_yolo\labels\val... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:02<00:00, 217.62it/s]

val: New cache created: C:\Users\lynxv\Desktop\Faculdade\Mestrado\2 Semestre\APVC\ProjectV2\chestx_det10_yolo\labels\val.cache



c:\Users\lynxv\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [04:31<00:00,  8.00s/it]


                   all        542       1476      0.555      0.393      0.406      0.201
           Atelectasis         48         51      0.381      0.314      0.317       0.11
         Calcification         38         67      0.594      0.299      0.321     0.0931
         Consolidation        289        446      0.616      0.518      0.544      0.261
              Effusion        252        372      0.533      0.478      0.448      0.183
             Emphysema         39         66      0.629      0.803      0.756      0.583
              Fibrosis         82        120      0.449      0.367      0.386      0.201
              Fracture         76        115       0.68       0.33      0.373      0.128
                  Mass         30         31      0.716      0.258       0.32       0.18
                Nodule         77        166      0.474      0.277      0.301      0.138
          Pneumothorax         35         42      0.477      0.286      0.295      0.131
Speed: 3.2ms preproce

AttributeError: 'DetMetrics' object has no attribute 'items'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        names (Dict[int, str]): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (Dict[str, float]): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
        stats (Dict[str, List]): A dictionary containing lists for true positives, confidence scores, predicted classes, target classes, and target images.
        nt_per_class: Number of targets per class.
        nt_per_image: Number of targets per image.
    